In [1]:
import json
import numpy as np
import pandas as pd
import os

Download the dataset following https://github.com/lupantech/ScienceQA#ghost-download-the-dataset.\
We only need the test set.

In [5]:
with open("./ScienceQA/data/scienceqa/problems.json", "r") as f:
    data = json.load(f)

test_list = [{id: item} for id, item in data.items() if item['split'] == 'test' and (item['image'] != 'null' and item['image'] is not None)]
len(test_list)

2017

In [6]:
def group_by_choices(data_list):
    grouped_by_choices = {}
    for d in data_list:
        for key, value in d.items():
            num_choices = len(value['choices'])
            if num_choices not in grouped_by_choices:
                grouped_by_choices[num_choices] = []
            
            # Create a new dictionary with choices as separate keys
            new_dict = value.copy()
            for i, choice in enumerate(value['choices']):
                new_dict[f'choice{i+1}'] = choice
            new_dict['image_id'] = key
            new_dict['image_path'] = f"{value['split']}/{key}/image.png"
            new_dict['label'] = chr(65 + value['answer'])
            new_dict.pop('choices')
            
            grouped_by_choices[num_choices].append(new_dict)
    return grouped_by_choices

In [7]:
!mkdir ScienceQA/meta_data

In [8]:
dataframes = {}
test_grouped_by_choices = group_by_choices(test_list)
for num_choices, questions in test_grouped_by_choices.items():
    dataframes[num_choices] = pd.DataFrame(questions)
for num_choices, df in dataframes.items():
    df.to_csv(f'ScienceQA/meta_data/{num_choices}choices_test.csv', index=False)